# Neuro ML 2020

## Seminar 5: Functional connectivity

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

import nilearn # pip install nilearn
import networkx as nx # pip install networkx
import diagram2vec # pip install diagram2vec

## Data

In [ ]:
# load ABIDE1 data from NYU site
ts = np.load("./data/TS_R_NYU.npy")
(n_patients, n_steps, n_regions) = ts.shape

### Multivariate time-series

#### Visualization of time series

In [ ]:
# visualization of raw time series
plt.figure(figsize=(16.5,5))
plt.title("First 3 time series")
plt.hlines(0, -2, n_steps+2, linewidth=1.0, linestyles="dotted")
plt.plot(ts[0,:,:3])
plt.show()

In [ ]:
# mean and standard deviation
print("Mean ± std of the:")
print("1st time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,0]), np.std(ts[0,:,0])))
print("2nd time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,1]), np.std(ts[0,:,1])))
print("3rd time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,2]), np.std(ts[0,:,2])))

#### Normalization and trend removal

In [ ]:
# break 2nd and 3rd time series
ts[0,:,1] = ts[0,:,1] + 150 # add mean shift
ts[0,:,2] = ts[0,:,2] + np.linspace(0, 150, n_steps) # add trend

In [ ]:
print("Mean ± std of the:")
print("1st time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,0]), np.std(ts[0,:,0])))
print("2nd time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,1]), np.std(ts[0,:,1])))
print("3rd time series: {:.3f} ± {:.3f}".format(np.mean(ts[0,:,2]), np.std(ts[0,:,2])))

In [ ]:
# visualization of broken time series
plt.figure(figsize=(16.5,5))
plt.title("First 3 time series")
plt.hlines(0, -2, n_steps+2, linewidth=1.0, linestyles="dotted")
plt.plot(ts[0,:,:3])
plt.show()

In [ ]:
from nilearn.signal import clean

In [ ]:
ts_normalized = np.zeros_like(ts)

# normalize and detrend
for i in range(ts.shape[0]):
    ts_normalized[i] = clean(ts[i], standardize="zscore", detrend=True)
    
print("Mean ± std of the:")
print("1st time series: {:.3f} ± {:.3f}".format(np.mean(ts_normalized[0,:,0]), np.std(ts_normalized[0,:,0])))
print("2nd time series: {:.3f} ± {:.3f}".format(np.mean(ts_normalized[0,:,1]), np.std(ts_normalized[0,:,1])))
print("3rd time series: {:.3f} ± {:.3f}".format(np.mean(ts_normalized[0,:,2]), np.std(ts_normalized[0,:,2])))

In [ ]:
# visualization of raw time series
plt.figure(figsize=(16.5,5))
plt.title("First 3 time series")
plt.hlines(0, -2, n_steps+2, linewidth=1.0, linestyles="dotted")
plt.plot(ts_normalized[0,:,:3])
plt.show()

## Metrics of functional connectivity

### Pearson correlation

In [ ]:
from nilearn.connectome import ConnectivityMeasure
from sklearn.covariance import EmpiricalCovariance

covariance_estimator = EmpiricalCovariance()
connectivity_correlation = ConnectivityMeasure(kind="correlation", cov_estimator=covariance_estimator)

In [ ]:
ts[0].shape

In [ ]:
R = connectivity_correlation.fit_transform(ts)
R.shape

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16.5,5))
plt.suptitle("Correlation connectivity matrices")
ax1.imshow(R[0])
ax2.imshow(R[1])
ax3.imshow(R[2])
plt.show()

### Regularization

_Condition number_ - minimum/maximum eigenvalue ratio of a matrix


#### Tikhonov regularization

$$\tilde{\mathbf{C}}_X = \mathbf{C}_X + \alpha \mathbf{I},~~~\alpha > 0$$

**Task**

Check the minimum eigenvalue of a correlation matrix

In [ ]:
np.min(np.linalg.eigvalsh(R[0]))

**Task**

Apply Tikhonov regularization and check how it affects the minimum eigenvalue of a correlation matrix

In [ ]:
### your code here

#### Shrinkage estimators

Ledoit-Wolf

$$\tilde{\mathbf{C}}_X = (1 - \beta)\mathbf{C}_X + \alpha \beta \mathbf{I},~~~\alpha > 0, 0 \leq \beta \leq 1\\
\alpha = \frac{trace(\mathbf{C})}{n_{features}}$$

A well conditioned estimator for large dimensional covariance matrices, $\alpha$ is predefined according to formula, $\beta$ is inferred from data.

In [ ]:
from sklearn.covariance import LedoitWolf

cov_estimator_shrinked = LedoitWolf()
connectivity_correlation_shrinked = ConnectivityMeasure(kind="correlation", cov_estimator=cov_estimator_shrinked)

In [ ]:
# check the value of beta parameter
cov = cov_estimator_shrinked.fit(ts[0])
cov.shrinkage_

In [ ]:
R_shrinked = connectivity_correlation_shrinked.fit_transform(ts)

In [ ]:
# checking the minimum eigenvalue
np.min(np.linalg.eigvalsh(R_shrinked[0]))

### Spearman correlation

In [ ]:
from scipy.stats import spearmanr

In [ ]:
S = np.zeros((3, n_regions, n_regions))

for i in range(3):
    S[i], _ = spearmanr(ts[i])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16.5,5))
plt.suptitle("Spearman correlation connectivity matrices")
ax1.imshow(S[0])
ax2.imshow(S[1])
ax3.imshow(S[2])
plt.show()

### Mutual information

Mutual information measures the information that random variables $X$ and $Y$ share, how much knowing one of these variables reduces uncertainty about the other. determined how different to joint distributon $p(X, Y)$ is to the production of the marginal distrubutions $p(X) p(Y)$.

$$I(X, Y) = \sum_{(x, y)} p(x, y) \log_2 \left( \frac{p(x, y)}{p(x)p(y)} \right)$$

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(ts[2,:,0], ts[2,:,12], ".")

In [ ]:
from sklearn.metrics import mutual_info_score

def calc_MI(x, y, bins=10):
    c_xy = np.histogram2d(x, y, bins)[0]
    mi = mutual_info_score(None, None, contingency=c_xy)
    return mi

def bound(x):
    return np.sqrt(1 - np.exp(-2 * x))

In [ ]:
l = calc_MI(ts[1,:,42], ts[1,:,45])
l, bound(l)

In [ ]:
c_xy = np.rot90(np.histogram2d(ts[2,:,0], ts[2,:,12], 10)[0])
plt.imshow(c_xy)

In [ ]:
l = calc_MI(ts[1,:,42], ts[1,:,41])
l, bound(l)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(ts[2,:,42], ts[2,:,41], ".")

In [ ]:
c_xy = np.rot90(np.histogram2d(ts[2,:,42], ts[2,:,41], 10)[0])
plt.imshow(c_xy)

In [ ]:
%%time
M = np.zeros((3, n_regions, n_regions))

for k in range(3):
    for i in range(n_regions):
        for j in range(i, n_regions):
            M[k,i,j] = bound(calc_MI(ts[k,:,i], ts[k,:,j]))
            
    M[k] = M[k] + M[k].T
    np.fill_diagonal(M[k], 1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16.5,5))
plt.suptitle("Mutual information connectivity matrices")
ax1.imshow(M[0])
ax2.imshow(M[1])
ax3.imshow(M[2])
plt.show()



#### Thresholding

In [ ]:
R_tresholded = R[0].copy()
np.fill_diagonal(R_tresholded, 0)
R_tresholded[R_tresholded < 0.5] = 0.0
R_tresholded

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(5,5))
plt.suptitle("Thresholded Pearson correlation matrix")
ax1.imshow(R_tresholded)
plt.show()

## Network visualization

In [ ]:
from nilearn import plotting

# get coordinates of brain regions
atlas_aal = nilearn.datasets.fetch_atlas_aal()
coordinates = plotting.find_parcellation_cut_coords(labels_img=atlas_aal["maps"])

In [ ]:
# nilearn graph drawing
fig = plt.figure(figsize=(13,6))
edge_options = {"color": "r", "linewidth": 1.5, "alpha": 0.5}
plotting.plot_connectome(R_tresholded, coordinates, figure=fig, edge_kwargs=edge_options)

In [ ]:
# matplotlib graph drawing
fig, (ax1) = plt.subplots(1, 1, figsize=(8,8))
ax1.set_title("Correlation graph")
nx.draw_shell(nx.from_numpy_array(R_tresholded), ax=ax1)
plt.show()

## Network analysis

### Graph-theoretic

In [ ]:
# create graph from connectivity matrix
G_R = nx.from_numpy_array(R_tresholded)

#### Node degree

In [ ]:
degree = np.array([degree[1] for degree in nx.degree(G_R)])
degree

In [ ]:
neighbor_degree_avg = np.array(list(nx.average_neighbor_degree(G_R).values()))
neighbor_degree_avg

#### Centralities

In [ ]:
centrality_betweenness = np.array(list(nx.betweenness_centrality(G_R).values()))
centrality_betweenness

In [ ]:
centrality_closeness = np.array(list(nx.closeness_centrality(G_R).values()))
centrality_closeness

#### Clustering coefficient

In [ ]:
clustering_coefficient_local = np.array(list(nx.clustering(G_R).values()))
clustering_coefficient_local

#### Efficiency

In [ ]:
nx.local_efficiency(G_R)

In [ ]:
nx.global_efficiency(G_R)

## Spectral graph theory

Eigenvalues of
- connectivity matrix
- Laplacian matrix

#### Spectrum

Solve for $\mathbf{\lambda}$ the eigenvalue problem, where $\mathbf{A}$ is the connectivity matrix

$$\mathbf{Av} = \mathbf{\lambda} \mathbf{v}$$

In [ ]:
eigenvalues, _ = np.linalg.eigh(R[0])
eigenvalues

#### Laplacian spectrum

Solve for $\mathbf{\lambda}$ the eigenvalue problem

$$\mathbf{Lv} = \mathbf{\lambda} \mathbf{v},$$

where $\mathbf{L}$ is the Laplacian matrix of the graph given by the connectivity matrix $\mathbf{A}$

$$\mathbf{L} = \mathbf{D} - \mathbf{A}$$

In [ ]:
# laplacian matrix L = D - A
# A is contained in variable R_tresholded

# D, your code here

# L, your code here

In [ ]:
eigenvalues_laplacian, _ = np.linalg.eigh(L)
eigenvalues_laplacian

## Topological

Loops, Betti numbers, persistent homology

In [ ]:
from ripser import ripser
import diagram2vec

In [ ]:
# reverse matrix to add higher correlated edges first to the filtration
R_filtered = 1 - np.abs(R[0])

In [ ]:
# compute persistence diagram of the network
diagram_R = ripser(R_filtered, distance_matrix=True)["dgms"]
diagram_R

In [ ]:
# vectorize persistent diagram
betti_curve = diagram2vec.persistence_curve(diagram_R)
betti_curve

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16.5,5))
ax1.set_title("Persistence diagram")
ax1.set_xlim(-0.025,1)
ax1.set_ylim(-0.025,1)
ax1.scatter(diagram_R[0][:,0], diagram_R[0][:,1], c="b")
ax1.scatter(diagram_R[1][:,0], diagram_R[1][:,1], c="r")
ax2.set_title("0th Betti number curve")
ax2.plot(betti_curve[0,0], c="b")
ax3.set_title("1st Betti number curve")
ax3.plot(betti_curve[0,1], c="r")
plt.show()

## Machine learning

Use the computed graph, spectral and topological classes features with sklearn classifiers. Try concatenating and/or boosting features of different classes, and stacking/emsembling of classifiers.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# load ABIDE1 data from NYU site
ts = np.load("./data/TS_R_NYU.npy")
y = np.load("./data/y_nyu.npy")

In [ ]:
# compute correlation networks
R = ConnectivityMeasure(kind="correlation").fit_transform(ts)
R.shape

In [ ]:
R.shape

#### Topological features

In [ ]:
# topological features
R_filtration = 1 - np.abs(R)

diagrams = []

for i, R_filtered in enumerate(R_filtration):
    diagram = ripser(R_filtered, distance_matrix=True)["dgms"]
    diagrams.append(diagram)

In [ ]:
X_topological = diagram2vec.persistence_curve(diagrams, quantity="persistence", m=40)

In [ ]:
X_topological[:,0].shape

In [ ]:
clf = DecisionTreeClassifier(random_state=42, max_depth=5)
cross_val_score(clf, X_topological[:,1], y, cv=10).mean()

#### Graph features

In [ ]:
R_thresholded = np.copy(R)
for i in range(R_thresholded.shape[0]):
    np.fill_diagonal(R_thresholded[i], 0)

R_thresholded[R_thresholded < 0.47] = 0.0

In [ ]:
# graph features
X_graph = np.zeros((n_patients, n_regions*5))

for i in range(R_thresholded.shape[0]):
    
    G = nx.from_numpy_array(R_thresholded[i])
    
    degree = np.array([degree[1] for degree in nx.degree(G)])
    neighbor_degree_avg = np.array(list(nx.average_neighbor_degree(G).values()))
    centrality_betweenness = np.array(list(nx.betweenness_centrality(G).values()))
    centrality_closeness = np.array(list(nx.closeness_centrality(G).values()))
    clustering_coefficient = np.array(list(nx.clustering(G).values()))
    
    feature_i = np.concatenate((degree, neighbor_degree_avg, centrality_betweenness, centrality_closeness, clustering_coefficient))
    X_graph[i] = feature_i


In [ ]:
cross_val_score(clf, X_graph, y, cv=10).mean()

#### Spectral features

In [ ]:
# spectral features (eigher spectrum or Laplacian spectrum)
X_spectral = np.zeros((n_patients, n_regions))

for i in range(R_thresholded.shape[0]):
    eigenvalues_i, _ = np.linalg.eigh(R_thresholded[i])
    X_spectral[i] = eigenvalues_i

In [ ]:
cross_val_score(clf, X_spectral, y, cv=10).mean()

#### Features concatenation

In [ ]:
cross_val_score(clf, np.concatenate((X_graph, X_spectral, X_topological[:,1]), axis=1), y, cv=10).mean()

**Task**

Repeat the feature extraction and machine learning pipeline on mutual information matrices. Note that the estimation of mutual information matrices could be be time-consuming.